# Reseach Topic: Predicting Wage with Machline Methods
## What am I doing ?
For now, we predict salary with several models.

## Data
The source of data is NLSY97. The portal can be accessed from [here](https://www.nlsinfo.org/investigator/pages/search.jsp?s=NLSY97). The data set is pre-cleaned by STATA with around 4500 obervations each year from 2. Features include age, year of experience, gender, schooling, race, marital status, industry, region(not yet!!). 

## Run Some Models
Bofore running anything, we split the data into training set, validation set, test set. <br/> I am preparing to do the following models:
- Linear model (Panel Data Regression)
- Trees and Forests (Regression Trees, Random Forests and XGBoost)
- Neural Nets (Multilayer Perceptrons)

The performences of models will be compared in accuracy and R-square

## Further Thoughts
(Haven't done yet) We resample the data into previous years and the following years. Then we train the model on previous years data and test it on the data from the following years.

# Let's get started :D

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

'''
Read in Data
'''
data_path = './data/mincer.xlsx'
dat = pd.read_excel(data_path)
dat = dat.fillna(0)

In [0]:
years = [i for i in range(2005,2011)]+[2013]+[2015]
dat_tmp = dat 
train, test =  train_test_split(dat_tmp, test_size = 0.2, random_state = 12)
y_train, X_train, y_test, X_test = train['lnwage'], train.iloc[:,4:], test['lnwage'], test.iloc[:,4:]

##  Explore the dataset

In [0]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style = "whitegrid")

sns.distplot(dat[dat['year']==2005]['age'],kde = False)

In [0]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
f, axes = plt.subplots(2, 4, figsize=(8, 4), sharex=True)

i,j = 0,0
for y in years:
    sns.distplot(dat[dat['year']==y]['lnwage'], ax = axes[j,i])
    if i < 3:
        i += 1
    else:
        i,j = 0,1 
plt.setp(axes, yticks=[])
plt.tight_layout()

## Define Models here

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.optimizers import adam

def mincer(X_train, y_train,  X_test, y_test):
    regressor = LinearRegression()
    reg = regressor.fit(X_train,y_train)
    return evaluate(reg, X_train, X_test, y_train, y_test)

def trees(X_train, y_train, X_test, y_test,model="rf", n_estimators = 100,max_depth=15):
    if  model == "gb":
        regressor = GradientBoostingRegressor(max_depth=max_depth, n_estimators = n_estimators)
    else:
        regressor = RandomForestRegressor(max_depth, random_state=1)
    reg = regressor.fit(X_train, y_train)
    return evaluate(reg, X_train, X_test, y_train, y_test)


def tree(X_train, y_train, X_test, y_test, max_depth = 3):
    regressor = DecisionTreeRegressor(max_depth=max_depth)
    reg = regressor.fit(X_train, y_train)
    return evaluate(reg, X_train, X_test, y_train, y_test)

def nn(X_train, y_train, X_test, y_test, verbose = 1):
    model = Sequential()
    model.add(Dense(8, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal',activation='relu'))
    # model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    # model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    model.add(Dense(1, kernel_initializer='normal',activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    history = model.fit(X_train, y_train, epochs=10, batch_size = 50,  verbose = verbose, validation_split=0.10)
    
    return evaluate(model, X_train, X_test, y_train, y_test)

def evaluate(model, X_train, X_test, y_train, y_test):
    y_pred_in = np.array( model.predict(X_train) )
    y_pred_out = np.array( model.predict(X_test) )
    if y_pred_in.ndim >1:
        y_pred_in = y_pred_in.flatten()
        y_pred_out =  y_pred_out.flatten()

    r2_in = metrics.r2_score(y_train,y_pred_in)
    r2_out = metrics.r2_score(y_test,y_pred_out)
    mse_in = np.mean((y_pred_in - np.array(y_train))**2)
    mse_out = np.mean((y_pred_out - np.array( y_test) )**2 )

    return mse_in, mse_out, r2_in, r2_out

# Test 1: Comparasion Between the Models

## Classification And Regression Trees (CART)

In [0]:
from sklearn import tree
plt.figure()
regressor = DecisionTreeRegressor(max_depth=3)
reg = regressor.fit(X_train, y_train)
tree.plot_tree(reg, filled= True)
plt.show()
# plt.savefig('./image_output/tree.png')

## Random Forests

In [0]:
regressor = RandomForestRegressor(max_depth = 10, random_state=1)
forest = regressor.fit(X_train, y_train)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
n_features = X_train.shape[1]
# n_features = 10
# Print the feature ranking
print("Feature ranking:")

for f in range(n_features ):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(n_features ), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(n_features), indices)
plt.xlim([-1, n_features])
plt.show()

In [0]:
trees(X_train, y_train, X_test, y_test, model="rf",max_depth=40)

In [11]:
# Brute force the optimal depth for CART and Random Forest
rf, tr =[],[] 
idx = [i for i in range(1,40)]
for dp in idx:
    rf.append(trees(X_train, y_train, X_test, y_test, model="rf",max_depth=dp))
    tr.append(tree(X_train, y_train, X_test, y_test, max_depth=dp))

rf_mse_out = np.array(rf)[:,1]
rf_r2_out = np.array(rf)[:,3]
tree_mse_out = np.array(tr)[:,1]
tree_r2_out = np.array(tr)[:,3]

plt.plot(idx, rf_mse_out, label = "Random Forest")
plt.plot(idx, tree_mse_out, label = 'Regression Tree')
plt.xlabel("max_depth")
plt.ylabel("Mean Squared Error")
plt.legend()
plt.show()

plt.plot(idx, rf_r2_out, label = "Random Forest")
plt.plot(idx, tree_r2_out, label = 'Regression Tree')
plt.xlabel("max_depth")
plt.ylabel(r'$R^2$')
plt.legend()
plt.show()

TypeError: 'module' object is not callable

## Gradient Boosting Machine

In [0]:
regressor = GradientBoostingRegressor(max_depth=5)
boost= regressor.fit(X_train, y_train)
importances = boost.feature_importances_
plt.bar(range(len(boost.feature_importances_)), boost.feature_importances_)
plt.show()

In [0]:
# Brute force the optimal depth for CART and Random Forest
a, b, c =[],[],[] 
idx = [i for i in range(1,20)]
for dp in idx:
    a.append(trees(X_train, y_train, X_test, y_test, model="gb",n_estimators = 50,max_depth=dp))
    b.append(trees(X_train, y_train, X_test, y_test, model="gb",n_estimators = 100,max_depth=dp))
    c.append(trees(X_train, y_train, X_test, y_test, model="gb",n_estimators = 200,max_depth=dp))
a = np.array(a)[:,1]
b = np.array(b)[:,1]
c = np.array(c)[:,1]

plt.plot(idx, a, label = "n_estimators = 50")
plt.plot(idx, b, label = 'n_estimators = 100')
plt.plot(idx, c, label = 'n_estimators = 200')

plt.xlabel("max_depth")
plt.ylabel("Mean Squared Error")
plt.legend()
plt.show()

In [0]:
plt.plot(idx, a, label = "n_estimators = 50")
plt.plot(idx, b, label = 'n_estimators = 100')
plt.plot(idx, c, label = 'n_estimators = 200')

plt.xlabel("max_depth")
plt.ylabel(r'$R^2$')
plt.legend()

## Neural Net

In [0]:
model = Sequential()
model.add(Dense(128, input_dim=29, kernel_initializer='normal', activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=50, batch_size = 32,  verbose=1, validation_split=0.2)



In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Model accuracy')
plt.ylabel('Accuracy (MAE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Overall comparation Run 4 models and take the averges

In [0]:
iters = 5
res_linear, res_tree, res_rf, res_xgb, res_nn = np.array([0.0]*4),np.array([0.0]*4),np.array([0.0]*4),np.array([0.0]*4),np.array([0.0]*4)
for i in range(iters):
    print("foler:{}".format(i+1))
    res_linear += np.array(mincer(X_train, y_train, X_test, y_test) )

    res_tree += np.array( tree(X_train, y_train, X_test, y_test) )

    res_rf += np.array( trees(X_train, y_train, X_test, y_test ) )

    res_xgb += np.array( trees(X_train, y_train, X_test, y_test,model = "gb",max_depth=12) )

    res_nn += np.array( nn(X_train, y_train, X_test, y_test) )

tab1 = "Regression & "+" & ".join([str(round(num,5)) for num in res_linear/iters ]) + r" \\" + "\n" 
tab2 = "Regression Tree & "+" & ".join([str(round(num,5)) for num in res_tree/iters ]) + r" \\" + "\n"
tab3 = "Random Forest & "+" & ".join([str(round(num,5)) for num in res_rf/iters]) +r" \\" +  "\n"
tab4 = "XGBoost & "+" & ".join([str(round(num,5)) for num in res_xgb/iters]) + r" \\" + "\n"
tab5 = "Neural Networks & "+" & ".join([str(round(num,5)) for num in res_nn/iters]) + r" \\" + "\n"
print( tab1+ tab2+ tab3+ tab4+ tab5)

# Test 2: Performance Over the Years
Now I am running the models over the years (2005 to 2011, 2013, 2015). 

In [0]:
years = [i for i in range(2005,2011)]+[2013]+[2015]
res={}
res['ols'],res['rf'],res['gbm'],res['trees'], res['nets'] = [],[],[],[],[]
for y in years:
    print("---------current test year is {}---------".format(y) )
    data = dat[dat['year']==y]
    train, test =  train_test_split(data, test_size = 0.25, random_state = 12)
    y_train, X_train, y_test, X_test = train['lnwage'], train.iloc[:,4:], test['lnwage'], test.iloc[:,4:]
    res['ols'].append( mincer(X_train, y_train,  X_test, y_test) )
    res['rf'].append( trees(X_train, y_train, X_test, y_test, model= "rf",max_depth=5) )
    res['gbm'].append( trees(X_train, y_train, X_test, y_test, model= "gb",max_depth=5) )
    nn_res = list(nn(X_train, y_train, X_test, y_test, verbose = 0))
    if abs(nn_res[3])>1: 
        nn[3] = 0
    res['nets'].append(nn_res)
    

In [0]:
import matplotlib.pyplot as plt
ylabels = ['MSE','MSE',r'$R^2$',r'$R^2$']
titles= ['MSE, training set',
         'MSE, testing set',
        r'$R^2$, training set',
        r'$R^2$, testing set'
        ]
for i in range(4):
    print(titles[i])
    plt.plot(years, np.array(res['ols'])[:,i], label = "Mincer")
    plt.plot(years, np.array(res['rf'])[:,i], label = "Random Forests")
    plt.plot(years, np.array(res['gbm'])[:,i], label = "Gradient Boosting Machine")
    plt.plot(years, np.array(res['nets'])[:,i], label = "Neural Nets")
    plt.ylabel(ylabels[i])
    plt.xlabel('year')
    plt.title(titles[i])
    plt.legend()
    plt.show()

### Plot activation functions

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import math

x = np.arange(-2, 2, .1)
zero = np.zeros(len(z))
a = np.max([zero, z], axis=0)
b = 1/(1+np.exp(-x))
c = np.tanh(x)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, a, label = "Relu")
ax.plot(x, b, label = "Sigmoid")
ax.plot(x,c,label = "tanh")
ax.set_ylim([-1.0, 1.0])
ax.set_xlim([-2.0, 2.0])
# ax.grid(True)
ax.set_xlabel('z')
ax.legend()
ax.set_title('Popular Activation Functions')

plt.show()